In [1]:
import torch
import torchvision
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/My Drive/Histopathology data/dualclass.zip" "dualclass.zip"

In [3]:
!unzip -q dualclass.zip

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
with open('dualclass/files_list.txt', 'r') as fp:
    files = fp.read().splitlines()
classes_str = [file.split('/')[0] for file in files]
classes_str, classes = np.unique(classes_str, return_inverse=True)
print(classes_str)
train_files, test_files, train_classes, test_classes = train_test_split(files, classes, train_size=0.7, test_size=0.3, stratify=classes)

['NORM' 'TUM']


In [5]:
from PIL import Image
import torch.utils.data as data
class Hist(data.Dataset):
    def __init__(self, root, files, classes, transforms=None):
        self.root = root
        self.files = files
        self.classes = classes
        self.transforms = transforms

    def __getitem__(self, item):
        image = Image.open(os.path.join(self.root, self.files[item])).convert(mode="RGB")
        class_ = self.classes[item]
        if self.transforms:
            image = self.transforms(image)
        return image, class_

    def __len__(self):
        return len(self.files)

In [6]:
batch_size_train = 5 # We use a small batch size here for training
batch_size_test = 10 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.Resize((224, 224)),
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#image datasets
train_dataset = Hist('dualclass/', train_files, train_classes, 
                         transforms=image_transform)
test_dataset = Hist('dualclass/', test_files, test_classes, 
                        transforms=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True, num_workers=2)

Label: NORM


In [7]:
# import library
import matplotlib.pyplot as plt
# We can check the dataloader
_, (example_datas, labels) = next(enumerate(train_loader))
sample = example_datas[0]
# show the data
plt.imshow(sample.permute(1, 2, 0));
print("Label: " + str(classes_str[labels[0]]))

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import torch.nn as nn
import torch.nn.functional as F


class InitBlock(nn.Module):
    expansion = 2  
    
  
    def __init__(self, in_planes, cardinality=32, bottleneck_width=4, stride=1):
        super(InitBlock, self).__init__()
        group_width = cardinality * bottleneck_width
        self.convolution1 = nn.Conv2d(in_planes, group_width, kernel_size=1, bias=False)
        self.batchnormal1 = nn.BatchNorm2d(group_width)
        
        self.convolution2 = nn.Conv2d(group_width, group_width, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.batchnormal2 = nn.BatchNorm2d(group_width)
        self.convolution3 = nn.Conv2d(group_width, self.expansion*group_width, kernel_size=1, bias=False)
        self.batchnormal3 = nn.BatchNorm2d(self.expansion*group_width)

        self.shortblock = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*group_width:
            self.shortblock = nn.Sequential(
              nn.Conv2d(in_planes, self.expansion*group_width, kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(self.expansion*group_width)
          )

    def forward(self, x):
        y = F.relu(self.batchnormal1(self.convolution1(x)))
        y = F.relu(self.batchnormal2(self.convolution2(y)))
        y = self.batchnormal3(self.convolution3(y))
        y += self.shortblock(x)
        y = F.relu(y)
        return y


class ResNeXt(nn.Module):
    def __init__(self, input_channel, num_blocks, cardinality, bottleneck_width, n_classes):
        super(ResNeXt, self).__init__()
        self.cardinality = cardinality
        self.bottleneck_width = bottleneck_width
        self.planes = 64

        self.convolution1 = nn.Conv2d(input_channel, 64, kernel_size=1, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.l1 = self.create_layer(num_blocks[0], 1)
        self.l2 = self.create_layer(num_blocks[1], 2)
        self.l3 = self.create_layer(num_blocks[2], 2)
       
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(cardinality*bottleneck_width*8, n_classes)

    def create_layer(self, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(InitBlock(self.planes, self.cardinality, self.bottleneck_width, stride))
            self.planes = InitBlock.expansion * self.cardinality * self.bottleneck_width
        
        self.bottleneck_width *= 2
        return nn.Sequential(*layers)

    def forward(self, x):
        y = F.relu(self.batchnorm1(self.convolution1(x)))
        y = self.l1(y)
        y = self.l2(y)
        y = self.l3(y)
        
        y = self.pool(y)
        y = y.view(y.size(0), -1)
        y = self.linear(y)
        return y



def ResNeXt29_2x64d(input_channel, n_classes):
    return ResNeXt(input_channel=input_channel, num_blocks=[3,3,3], cardinality=2, bottleneck_width=64, n_classes=n_classes)


In [16]:
import torch.nn.functional as F
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt29_2x64d(input_channel=3, n_classes=2).to(device)
learning_rate = 0.0002
weight_decay = 0.001
for param in model.parameters():
    param.requires_grad = True
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(parameters, lr=learning_rate, weight_decay=weight_decay)

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [17]:
from tqdm import tqdm_notebook as tqdm
##define train function
def train(model, device, train_loader, optimizer):
    # meter
    loss = AverageMeter()
    # switch to train mode
    model.train()
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    for batch_idx, (data, target) in enumerate(tk0):
        # after fetching the data transfer the model to the 
        # required device, in this example the device is gpu
        # transfer to gpu can also be done by 
        # data, target = data.cuda(), target.cuda()
        data, target = data.to(device), target.to(device)  
        # compute the forward pass
        # it can also be achieved by model.forward(data)
        output = model(data) 
        # compute the loss function
        loss_this = F.cross_entropy(output, target)
        # initialize the optimizer
        optimizer.zero_grad()
        # compute the backward pass
        loss_this.backward()
        # update the parameters
        optimizer.step()
        # update the loss meter 
        loss.update(loss_this.item(), target.shape[0])
    print('Train: Average loss: {:.4f}\n'.format(loss.avg))
    return loss.avg
        
##define test function
def test(model, device, test_loader):
    # meters
    loss = AverageMeter()
    acc = AverageMeter()
    correct = 0
    # switch to test mode
    model.eval()
    for data, target in test_loader:
      
        data, target = data.to(device), target.to(device)  # data, target = data.cuda(), target.cuda()
        # since we dont need to backpropagate loss in testing,
        # we dont keep the gradient
        with torch.no_grad():
            # compute the forward pass
            # it can also be achieved by model.forward(data)
            output = model(data)
        # compute the loss function just for checking
        loss_this = F.cross_entropy(output, target) # sum up batch loss
        # get the index of the max log-probability
        pred = output.argmax(dim=1, keepdim=True) 
        # check which are correct
        correct_this = pred.eq(target.view_as(pred)).sum().item()
        # accumulate the correct ones
        correct += correct_this
        # compute accuracy
        acc_this = correct_this/target.shape[0]*100.0
        # update the loss and accuracy meter 
        acc.update(acc_this, target.shape[0])
        loss.update(loss_this.item(), target.shape[0])
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss.avg, correct, len(test_loader.dataset), acc.avg))
    return loss.avg,acc.avg

In [ ]:
import os.path
from os import path

num_epoch = 5
train_loss=[]
test_loss=[]
accuracy_test=[]
acc=0
for epoch in range(1, num_epoch + 1):

    print(epoch)
    tl=train(model, device, train_loader, optimizer)
    stl,acct=test(model, device, test_loader)
    if acc<acct:
      if path.exists("ResNext2categoriesmodel.pth"):
        os.remove("ResNext2categoriesmodel.pth")

      torch.save(model.state_dict(), 'ResNext2categoriesmodel.pth')
      acc=acct
    train_loss.append(tl)
    test_loss.append(stl)
    accuracy_test.append(acct)

!cp ResNext2categoriesmodel.pth '/content/drive/My Drive/Histopathology data/'

train_l=str(train_loss)
test_l=str(test_loss)
accuracy_t=str(accuracy_test)

output_file = open('training_accuracy.txt', 'w')

for t in train_l:
    output_file.write(t)

output_file.close()


output_file = open('test_loss.txt', 'w')

for t in test_l:
    output_file.write(t)

output_file.close()

output_file = open('accuracy_test.txt', 'w')

for t in accuracy_t:
    output_file.write(t)

output_file.close()



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4245 [00:00<?, ?it/s]

In [ ]:
import os.path
from os import path
!cp "/content/drive/My Drive/Histopathology data/ResNext2categoriesmodel.pth" "ResNext2categoriesmodel.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt29_2x64d(input_channel=3, n_classes=2)
model.load_state_dict(torch.load('ResNext2categoriesmodel.pth', map_location=device))
model=model.to(device)

num_epoch = 5
train_loss=[]
test_loss=[]
accuracy_test=[]
acc=0
for epoch in range(1, num_epoch + 1):

    print(epoch)
    tl=train(model, device, train_loader, optimizer)
    stl,acct=test(model, device, test_loader)
    if acc<acct:
      if path.exists("ResNext2categoriesmodel.pth"):
        os.remove("ResNext2categoriesmodel.pth")

      torch.save(model.state_dict(), 'ResNext2categoriesmodel.pth')
      acc=acct
    train_loss.append(tl)
    test_loss.append(stl)
    accuracy_test.append(acct)

!cp ResNext2categoriesmodel.pth '/content/drive/My Drive/Histopathology data/'

train_l=str(train_loss)
test_l=str(test_loss)
accuracy_t=str(accuracy_test)

output_file = open('training_accuracy.txt', 'w')

for t in train_l:
    output_file.write(t)

output_file.close()


output_file = open('test_loss.txt', 'w')

for t in test_l:
    output_file.write(t)

output_file.close()

output_file = open('accuracy_test.txt', 'w')

for t in accuracy_t:
    output_file.write(t)

output_file.close()



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1414

Test: Average loss: 0.1472, Accuracy: 8570/9096 (94.22%)

2


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1417

Test: Average loss: 0.1161, Accuracy: 8747/9096 (96.16%)

3


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1440

Test: Average loss: 0.1207, Accuracy: 8698/9096 (95.62%)

4


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1431

Test: Average loss: 0.1122, Accuracy: 8739/9096 (96.08%)

5


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1399

Test: Average loss: 0.1149, Accuracy: 8738/9096 (96.06%)



In [ ]:
import os.path
from os import path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt29_2x64d(input_channel=3, n_classes=2)
model.load_state_dict(torch.load('ResNext2categoriesmodel.pth', map_location=device))
model=model.to(device)

num_epoch = 5
train_loss=[]
test_loss=[]
accuracy_test=[]
acc=0
for epoch in range(1, num_epoch + 1):

    print(epoch)
    tl=train(model, device, train_loader, optimizer)
    stl,acct=test(model, device, test_loader)
    if acc<acct:
      if path.exists("ResNext2categoriesmodel.pth"):
        os.remove("ResNext2categoriesmodel.pth")

      torch.save(model.state_dict(), 'ResNext2categoriesmodel.pth')
      acc=acct
    train_loss.append(tl)
    test_loss.append(stl)
    accuracy_test.append(acct)

!cp ResNext2categoriesmodel.pth '/content/drive/My Drive/Histopathology data/'

train_l=str(train_loss)
test_l=str(test_loss)
accuracy_t=str(accuracy_test)

output_file = open('training_accuracy.txt', 'w')

for t in train_l:
    output_file.write(t)

output_file.close()


output_file = open('test_loss.txt', 'w')

for t in test_l:
    output_file.write(t)

output_file.close()

output_file = open('accuracy_test.txt', 'w')

for t in accuracy_t:
    output_file.write(t)

output_file.close()



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1449

Test: Average loss: 0.1095, Accuracy: 8756/9096 (96.26%)

2


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1411

Test: Average loss: 0.1210, Accuracy: 8681/9096 (95.44%)

3


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1446

Test: Average loss: 0.1140, Accuracy: 8734/9096 (96.02%)

4


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1456

Test: Average loss: 0.1117, Accuracy: 8722/9096 (95.89%)

5


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1428

Test: Average loss: 0.1100, Accuracy: 8748/9096 (96.17%)



In [ ]:
!cp "/content/drive/My Drive/Histopathology data/ResNext2categoriesmodel.pth" "ResNext2categoriesmodel.pth"

In [ ]:
import os.path
from os import path



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt29_2x64d(input_channel=3, n_classes=2)
model.load_state_dict(torch.load('ResNext2categoriesmodel.pth', map_location=device))
model=model.to(device)

num_epoch = 10
train_loss=[]
test_loss=[]
accuracy_test=[]
acc=0
for epoch in range(1, num_epoch + 1):

    print(epoch)
    tl=train(model, device, train_loader, optimizer)
    stl,acct=test(model, device, test_loader)
    if acc<acct:
      if path.exists("ResNext2categoriesmodel.pth"):
        os.remove("ResNext2categoriesmodel.pth")

      torch.save(model.state_dict(), 'ResNext2categoriesmodel.pth')
      acc=acct
    train_loss.append(tl)
    test_loss.append(stl)
    accuracy_test.append(acct)

!cp ResNext2categoriesmodel.pth '/content/drive/My Drive/Histopathology data/'

train_l=str(train_loss)
test_l=str(test_loss)
accuracy_t=str(accuracy_test)

output_file = open('training_accuracy.txt', 'w')

for t in train_l:
    output_file.write(t)

output_file.close()


output_file = open('test_loss.txt', 'w')

for t in test_l:
    output_file.write(t)

output_file.close()

output_file = open('accuracy_test.txt', 'w')

for t in accuracy_t:
    output_file.write(t)

output_file.close()



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1439

Test: Average loss: 0.1171, Accuracy: 8712/9096 (95.78%)

2


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1437

Test: Average loss: 0.1138, Accuracy: 8733/9096 (96.01%)

3


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1449

Test: Average loss: 0.1155, Accuracy: 8740/9096 (96.09%)

4


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1462

Test: Average loss: 0.1138, Accuracy: 8734/9096 (96.02%)

5


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1434

Test: Average loss: 0.1427, Accuracy: 8605/9096 (94.60%)

6


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1465

Test: Average loss: 0.1246, Accuracy: 8693/9096 (95.57%)

7


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1430

Test: Average loss: 0.1479, Accuracy: 8574/9096 (94.26%)

8


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1421

Test: Average loss: 0.1210, Accuracy: 8655/9096 (95.15%)

9


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1401

Test: Average loss: 0.2030, Accuracy: 8384/9096 (92.17%)

10


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1407

Test: Average loss: 0.1231, Accuracy: 8647/9096 (95.06%)



In [ ]:
import os.path
from os import path



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNeXt29_2x64d(input_channel=3, n_classes=2)
model.load_state_dict(torch.load('ResNext2categoriesmodel.pth', map_location=device))
model=model.to(device)

num_epoch = 5
train_loss=[]
test_loss=[]
accuracy_test=[]
acc=0
for epoch in range(1, num_epoch + 1):

    print(epoch)
    tl=train(model, device, train_loader, optimizer)
    stl,acct=test(model, device, test_loader)
    if acc<acct:
      if path.exists("ResNext2categoriesmodel.pth"):
        os.remove("ResNext2categoriesmodel.pth")

      torch.save(model.state_dict(), 'ResNext2categoriesmodel.pth')
      acc=acct
    train_loss.append(tl)
    test_loss.append(stl)
    accuracy_test.append(acct)

!cp ResNext2categoriesmodel.pth '/content/drive/My Drive/Histopathology data/'

train_l=str(train_loss)
test_l=str(test_loss)
accuracy_t=str(accuracy_test)

output_file = open('training_accuracy.txt', 'w')

for t in train_l:
    output_file.write(t)

output_file.close()


output_file = open('test_loss.txt', 'w')

for t in test_l:
    output_file.write(t)

output_file.close()

output_file = open('accuracy_test.txt', 'w')

for t in accuracy_t:
    output_file.write(t)

output_file.close()



1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1408

Test: Average loss: 0.1207, Accuracy: 8706/9096 (95.71%)

2


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1419

Test: Average loss: 0.1124, Accuracy: 8741/9096 (96.10%)

3


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1407

Test: Average loss: 0.1270, Accuracy: 8683/9096 (95.46%)

4


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1430

Test: Average loss: 0.1092, Accuracy: 8736/9096 (96.04%)

5


  0%|          | 0/4245 [00:00<?, ?it/s]

Train: Average loss: 0.1421

Test: Average loss: 0.1207, Accuracy: 8703/9096 (95.68%)

